In [80]:
import os
import io
import pandas as pd
import numpy as np
import zipfile
import nltk
import string

from zipfile import ZipFile
from skllm.config import SKLLMConfig
from skllm import ZeroShotGPTClassifier

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [91]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [59]:
# upload OpenAI API keys
os.environ["OPENAI_SECRET_KEY"] ="YOUR_API_KEY"
OPENAI_SECRET_KEY=os.environ["OPENAI_SECRET_KEY"]
SKLLMConfig.set_openai_key(OPENAI_SECRET_KEY)

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving metadados20231030.json to metadados20231030.json


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving 20231030.zip to 20231030 (1).zip


In [ ]:
df1 = pd.read_json('metadados20231030.json')

In [ ]:
df1

,seqDocumento,dataPublicacao,tipoDocumento,numeroRegistro,processo,dataRecebimento,dataDistribuicao,ministro,recurso,teor,descricaoMonocratica,assuntos
0,215357135,1698634800000,ACÓRDÃO,202301057462,REsp 2062315,1680145200000,1682305200000,NANCY ANDRIGHI,None,Concedendo,None,4960;4960;9163
1,215057679,1698634800000,ACÓRDÃO,202302672600,HC 842135,1690513200000,1690513200000,ROGERIO SCHIETTI CRUZ,None,Concedendo,None,3608;3608;3608
2,215057670,1698634800000,ACÓRDÃO,202303084573,HC 849921,1692932400000,1693191600000,ROGERIO SCHIETTI CRUZ,None,Concedendo,None,4355;4355;3608;4355
3,213812340,1698634800000,ACÓRDÃO,202301573555,REsp 2070794,1683860400000,1684983600000,REYNALDO SOARES DA FONSECA,AGRAVO REGIMENTAL,Negando,None,3608;3608
4,215058245,1698634800000,ACÓRDÃO,202302257775,AREsp 2400347,1688007600000,1689217200000,ROGERIO SCHIETTI CRUZ,AGRAVO REGIMENTAL,Negando,None,3548;3548
...,...,...,...,...,...,...,...,...,...,...,...,...
2908,215259456,1698634800000,DECISÃO,202303918409,HC 864755,1698202800000,1698289200000,REYNALDO SOARES DA FONSECA,None,Não Conhecendo,Não conhecido o Habeas Corpus\r\n,3372;3372;3372
2909,215374404,1698634800000,DECISÃO,202303918345,HC 864622,1698202800000,1698289200000,JESUÍNO RISSATO (DESEMBARGADOR CONVOCADO DO TJ...,None,Não Conhecendo,Não conhecido o Habeas Corpus\r\n,3608;3608;3608
2910,215341365,1698634800000,DECISÃO,202300908592,AREsp 2321022,1679367600000,1689908400000,MARIA THEREZA DE ASSIS MOURA,PETIÇÃO,None,Proferido despacho de mero expediente determin...,9582;9582
2911,215258390,1698634800000,DECISÃO,201602244973,EAREsp 972562,1471489200000,1471834800000,OG FERNANDES,EMBARGOS DE DECLARAÇÃO,None,Proferido despacho de mero expediente nada def...,7895;7895;4703


In [26]:
df1.loc[(df1['seqDocumento'] == 215357135), 'text'] = 'some_string'

In [43]:
df1['text'][0]

'EMENTA\nRECURSO ESPECIAL. AÇÃO DE EXECUÇÃO DE TÍTULO EXTRAJUDICIAL. IMPENHORABILIDADE BEM DE FAMÍIA. AVERBAÇÃO DA PENHORA. REGISTRO IMOBILIÁRIO. OMISSÃO. CONTRADIÇÃO. NÃO DEMONSTRADA. AUSÊNCIA. DIVERGÊNCIA JURISPRUDENCIAL.\n1. Ação de execução de título extrajudicial ajuizada em 24/05/2022, da qual foram extraídos os presentes recursos especiais, interpostos em 26/01/2023 e conclusos ao gabinete em 24/04/2023.\n2. O propósito recursal é decidir se, vedada a expropriação, é possível a averbação da penhora no registro de imóveis do bem de família.\n3. A impenhorabilidade do bem de família não significa somente que o bem não pode ser expropriado para satisfação do credor. A determinação do art. 1º da Lei 8.009/90 implica em reconhecer que, no processo executório, o bem de família nem mesmo pode ser indicado à penhora.\n4. A penhora de bem de família é ato inválido, que não se perfectibiliza e, por conseguinte, não pode ter consequências para o mundo jurídico, não havendo que se falar em 

In [ ]:
zip_file = ZipFile('20231030.zip')
df2 = {text_file.filename: pd.read_csv(zip_file.open(text_file.filename),  lineterminator='\n')
       for text_file in zip_file.infolist()
       if text_file.filename.endswith('.txt')}

In [48]:
for i in range(len(df1)):
  with zipfile.ZipFile("20231030.zip") as zf:
    seqDoc = df1.iloc[i]['seqDocumento']
    filename = "20231030/" + str(seqDoc) + ".txt"
    with io.TextIOWrapper(zf.open(filename), encoding="utf-8") as f:
      txt = ""
      for line in f:
        txt += line
        # i += 1
        # if i > 10:
          # break
      # print(txt)
      df1.loc[(df1['seqDocumento'] == seqDoc), 'text'] = txt
      # break


In [37]:
with zipfile.ZipFile('20231030.zip') as z:
  i = 0
  for filename in z.namelist():
    print(filename)
    i += 1
    if i > 10:
      break
        # if not os.path.isdir(filename):
        #     # read the file
        #     with z.open(filename) as f:
        #         for line in f:
        #             print line

20231030/123356119.txt
20231030/133182923.txt
20231030/135120440.txt
20231030/168350693.txt
20231030/179256770.txt
20231030/179328124.txt
20231030/181376505.txt
20231030/182600722.txt
20231030/185109508.txt
20231030/186000580.txt
20231030/187164630.txt


In [51]:
df1.to_csv('20231030.csv', index=False, encoding='utf-8')

In [52]:
!cp 20231030.csv "drive/My Drive/"

In [70]:
df1['text'] = df1['text'].astype('str')

In [86]:
df1

,seqDocumento,dataPublicacao,tipoDocumento,numeroRegistro,processo,dataRecebimento,dataDistribuicao,ministro,recurso,teor,descricaoMonocratica,assuntos,text
0,215357135,1698634800000,ACÓRDÃO,202301057462,REsp 2062315,1680145200000,1682305200000,NANCY ANDRIGHI,None,Concedendo,None,4960;4960;9163,EMENTA\nRECURSO ESPECIAL. AÇÃO DE EXECUÇÃO DE ...
1,215057679,1698634800000,ACÓRDÃO,202302672600,HC 842135,1690513200000,1690513200000,ROGERIO SCHIETTI CRUZ,None,Concedendo,None,3608;3608;3608,ACÓRDÃO\nVistos e relatados estes autos em que...
2,215057670,1698634800000,ACÓRDÃO,202303084573,HC 849921,1692932400000,1693191600000,ROGERIO SCHIETTI CRUZ,None,Concedendo,None,4355;4355;3608;4355,ACÓRDÃO\nVistos e relatados estes autos em que...
3,213812340,1698634800000,ACÓRDÃO,202301573555,REsp 2070794,1683860400000,1684983600000,REYNALDO SOARES DA FONSECA,AGRAVO REGIMENTAL,Negando,None,3608;3608,ACÓRDÃO\nVistos e relatados estes autos em que...
4,215058245,1698634800000,ACÓRDÃO,202302257775,AREsp 2400347,1688007600000,1689217200000,ROGERIO SCHIETTI CRUZ,AGRAVO REGIMENTAL,Negando,None,3548;3548,ACÓRDÃO\nVistos e relatados estes autos em que...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2908,215259456,1698634800000,DECISÃO,202303918409,HC 864755,1698202800000,1698289200000,REYNALDO SOARES DA FONSECA,None,Não Conhecendo,Não conhecido o Habeas Corpus\r\n,3372;3372;3372,"DECISÃO<br>Cuida-se de habeas corpus, com pedi..."
2909,215374404,1698634800000,DECISÃO,202303918345,HC 864622,1698202800000,1698289200000,JESUÍNO RISSATO (DESEMBARGADOR CONVOCADO DO TJ...,None,Não Conhecendo,Não conhecido o Habeas Corpus\r\n,3608;3608;3608,"DECISÃO<br>Trata-se de HABEAS CORPUS, com pedi..."
2910,215341365,1698634800000,DECISÃO,202300908592,AREsp 2321022,1679367600000,1689908400000,MARIA THEREZA DE ASSIS MOURA,PETIÇÃO,None,Proferido despacho de mero expediente determin...,9582;9582,DECISÃO<br>Cuida-se de petição (n. 980812/2023...
2911,215258390,1698634800000,DECISÃO,201602244973,EAREsp 972562,1471489200000,1471834800000,OG FERNANDES,EMBARGOS DE DECLARAÇÃO,None,Proferido despacho de mero expediente nada def...,7895;7895;4703,"DECISÃO<br>Trata-se de petição nomeada como ""e..."


In [88]:
def pre_processing(df_original):
  df = df_original.copy()
  df['text'] = df['text'].str.lower()
  df['text'] = df.apply(lambda x: word_tokenize(x['text'], language='portuguese'), axis=1)
  table = str.maketrans('', '', string.punctuation)
  df['text'] = df.apply(lambda x: [w.translate(table) for w in x['text']], axis=1)
  df['text'] = df.apply(lambda x: [w for w in x['text'] if w.isalpha()], axis=1)
  stop_words = set(stopwords.words('portuguese'))
  df['text'] = df.apply(lambda x: [w for w in x['text'] if not w in stop_words], axis=1)
  df['text'] = df.apply(lambda x: ' '.join(x["text"]), axis=1)
  return df

In [92]:
df2 = pre_processing(df1)

In [93]:
df2

,seqDocumento,dataPublicacao,tipoDocumento,numeroRegistro,processo,dataRecebimento,dataDistribuicao,ministro,recurso,teor,descricaoMonocratica,assuntos,text
0,215357135,1698634800000,ACÓRDÃO,202301057462,REsp 2062315,1680145200000,1682305200000,NANCY ANDRIGHI,None,Concedendo,None,4960;4960;9163,ementa recurso especial ação execução título e...
1,215057679,1698634800000,ACÓRDÃO,202302672600,HC 842135,1690513200000,1690513200000,ROGERIO SCHIETTI CRUZ,None,Concedendo,None,3608;3608;3608,acórdão vistos relatados autos partes acima in...
2,215057670,1698634800000,ACÓRDÃO,202303084573,HC 849921,1692932400000,1693191600000,ROGERIO SCHIETTI CRUZ,None,Concedendo,None,4355;4355;3608;4355,acórdão vistos relatados autos partes acima in...
3,213812340,1698634800000,ACÓRDÃO,202301573555,REsp 2070794,1683860400000,1684983600000,REYNALDO SOARES DA FONSECA,AGRAVO REGIMENTAL,Negando,None,3608;3608,acórdão vistos relatados autos partes acima in...
4,215058245,1698634800000,ACÓRDÃO,202302257775,AREsp 2400347,1688007600000,1689217200000,ROGERIO SCHIETTI CRUZ,AGRAVO REGIMENTAL,Negando,None,3548;3548,acórdão vistos relatados autos partes acima in...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2908,215259456,1698634800000,DECISÃO,202303918409,HC 864755,1698202800000,1698289200000,REYNALDO SOARES DA FONSECA,None,Não Conhecendo,Não conhecido o Habeas Corpus\r\n,3372;3372;3372,decisão br cuidase habeas corpus pedido limina...
2909,215374404,1698634800000,DECISÃO,202303918345,HC 864622,1698202800000,1698289200000,JESUÍNO RISSATO (DESEMBARGADOR CONVOCADO DO TJ...,None,Não Conhecendo,Não conhecido o Habeas Corpus\r\n,3608;3608;3608,decisão br tratase habeas corpus pedido limina...
2910,215341365,1698634800000,DECISÃO,202300908592,AREsp 2321022,1679367600000,1689908400000,MARIA THEREZA DE ASSIS MOURA,PETIÇÃO,None,Proferido despacho de mero expediente determin...,9582;9582,decisão br cuidase petição n apresentada itape...
2911,215258390,1698634800000,DECISÃO,201602244973,EAREsp 972562,1471489200000,1471834800000,OG FERNANDES,EMBARGOS DE DECLARAÇÃO,None,Proferido despacho de mero expediente nada def...,7895;7895;4703,decisão br tratase petição nomeada embargos de...


In [94]:
df2.to_csv('20231030_new.csv', index=False, encoding='utf-8')

In [95]:
!cp 20231030_new.csv "drive/My Drive/"

In [97]:
# column to fit zero-shot classification
X = np.array(df2['text'])

In [99]:
X[0]

'ementa recurso especial ação execução título extrajudicial impenhorabilidade bem famíia averbação penhora registro imobiliário omissão contradição demonstrada ausência divergência jurisprudencial ação execução título extrajudicial ajuizada extraídos presentes recursos especiais interpostos conclusos gabinete propósito recursal decidir vedada expropriação possível averbação penhora registro imóveis bem família impenhorabilidade bem família significa somente bem pode expropriado satisfação credor determinação art lei implica reconhecer processo executório bem família pode indicado penhora penhora bem família ato inválido perfectibiliza conseguinte pode ter consequências mundo jurídico havendo falar expropriação inadmissível credor realize averbação penhora registro imobiliário bem família vedada expropriação vista penhora inválida desrespeitar norma ordem pública positivada lei ofensa art tribunal origem aplicando direito entende cabível hipótese soluciona integralmente controvérsia sub

In [100]:
clf = ZeroShotGPTClassifier(openai_model="gpt-3.5-turbo")
clf.fit(None, ["positive", "negative", "neutral"])
pred_gpt_3_5_turbo = pd.DataFrame(clf.predict(X), columns=['sentiment'])

  0%|          | 1/2913 [00:09<7:57:34,  9.84s/it]

Could not obtain the completion after 3 retries: `RateLimitError :: You exceeded your current quota, please check your plan and billing details.`
None
Could not extract the label from the completion: 'NoneType' object is not subscriptable


  0%|          | 2/2913 [00:19<7:55:10,  9.79s/it]

Could not obtain the completion after 3 retries: `RateLimitError :: You exceeded your current quota, please check your plan and billing details.`
None
Could not extract the label from the completion: 'NoneType' object is not subscriptable


  0%|          | 3/2913 [00:29<7:53:40,  9.77s/it]

Could not obtain the completion after 3 retries: `RateLimitError :: You exceeded your current quota, please check your plan and billing details.`
None
Could not extract the label from the completion: 'NoneType' object is not subscriptable


  0%|          | 3/2913 [00:34<9:23:24, 11.62s/it]


KeyboardInterrupt: ignored